https://gov.indexcoop.com/t/finance-nest-a-thread-of-monthly-financial-report-publications/3661/26

- Ethereum Investment Account: 0x462A63D4405A6462b157341A78Fd1baBfD3F8065
- Ethereum Operations Account: 0xFafd604d1CC8b6B3B6CC859cF80Fd902972371C1
- Ethereum Protocol Owned Liquidity Account: 0x3a36b94689f303aAf9BFE761068Efb8F78912023
- Polygon operations:  0xDECC5b013C74d8d3cEf1F8918768DDa17490B505

In [1]:
from credmark.cmf.ipython import create_cmf
from credmark.cmf.types import (Address, BlockNumber, Contract, Token, Portfolio, PriceWithQuote)
from credmark.cmf.types.data.fiat_currency_data import FIAT_CURRENCY_DATA_BY_SYMBOL
from credmark.cmf.model.errors import ModelRunError, ModelDataError
from web3 import HTTPProvider, Web3
from datetime import datetime, timezone, timedelta

cmf_param = {
                'block_number': None,
                'chain_to_provider_url': {'1': 'http://192.168.68.122:8545'},
                'api_url': 'http://192.168.68.122:8700',
                'use_local_models': '-', # use local to speed up
            }

# When using fill_db_evt in a loop, create cmf context outside the loop and sent _cmf_param = None
# When using fill_db_evt standalone, either 1) create cmf context outside the loop and sent _cmf_param = None, 2) sent in non-None cmf_param.
context, _model_loader = create_cmf(cmf_param)
context._web3 = Web3(HTTPProvider(context.web3.provider.endpoint_uri, request_kwargs={'timeout': 3600 * 10}))
context._web3.eth.default_block = int(context.block_number)
end_block = int(context.block_number)

In [2]:
import pandas as pd

# https://gov.indexcoop.com/t/finance-nest-a-thread-of-monthly-financial-report-publications/3661/26
# https://github.com/IndexCoop/address-book

df_addrs = pd.read_csv('https://raw.githubusercontent.com/IndexCoop/address-book/master/working-group-address-book.csv')

In [18]:
block_20221226 = BlockNumber.from_timestamp(datetime(2021, 12, 26, tzinfo=timezone.utc))
block_20221226

13877449

In [19]:
block_20221031 = BlockNumber.from_timestamp(datetime(2022, 10, 31,tzinfo=timezone.utc))
block_20220831 = BlockNumber.from_timestamp(datetime(2022, 8, 31,tzinfo=timezone.utc))

In [21]:
def pos_info(pos, blk):
    print(pos.asset.symbol)
    return (round(pos.amount * context.run_model('price.dex-db-prefer', {'address': pos.asset.address}, block_number=blk)['price'],1),
            pos.asset.symbol,
            pos.asset.address)

def index_coop_assets(_block_number):
    default_block = context.web3.eth.default_block
    context.web3.eth.default_block = _block_number
    try:
        eth_invest = '0x462A63D4405A6462b157341A78Fd1baBfD3F8065'
        eth_ops = '0xFafd604d1CC8b6B3B6CC859cF80Fd902972371C1'
        eth_liq = '0x3a36b94689f303aAf9BFE761068Efb8F78912023'

        index_coop_addrs = df_addrs.query('chainId == 1').address.to_list()
        index_coop_addrs = [eth_invest, eth_ops, eth_liq]

        position = context.run_model(
            'accounts.portfolio',
            {'accounts': index_coop_addrs},
            return_type = Portfolio,
            block_number = _block_number)

        poses = [
            [pos_info(pos, _block_number) for pos in position if pos.asset.address.checksum == Token('AAVE').address.checksum],
            [pos_info(pos, _block_number) for pos in position if pos.asset.address.checksum == Token('USDC').address.checksum],
            [pos_info(pos, _block_number) for pos in position if pos.asset.address.checksum == Token('DAI').address.checksum],
            [pos_info(pos, _block_number) for pos in position if pos.asset.address.checksum == Token('WETH').address.checksum],
            [pos_info(pos, _block_number) for pos in position if pos.asset.address.checksum == Token('0x5d3a536E4D6DbD6114cc1Ead35777bAB948E3643').address.checksum], # cDAI
            [pos_info(pos, _block_number) for pos in position if pos.asset.address.checksum == Token('0x39aa39c021dfbae8fac545936693ac917d5e7563').address.checksum], # cUSDC
            [pos_info(pos, _block_number) for pos in position if pos.asset.address.checksum == Token('0x33d63Ba1E57E54779F7dDAeaA7109349344cf5F1').address.checksum], # DATA 
            [pos_info(pos, _block_number) for pos in position if pos.asset.address.checksum == Token('0x47110d43175f7f2c2425e7d15792acc5817eb44f').address.checksum], # GMI
            [pos_info(pos, _block_number) for pos in position if pos.asset.address.checksum == Token('0x0954906da0Bf32d5479e25f46056d22f08464cab').address.checksum], # INDEX 
            [pos_info(pos, _block_number) for pos in position if pos.asset.address.checksum == Token('0x7C07F7aBe10CE8e33DC6C5aD68FE033085256A84').address.checksum], # icETH 
            [pos_info(pos, _block_number) for pos in position if pos.asset.address.checksum == Token('0x02e7ac540409d32c90bfb51114003a9e1ff0249c').address.checksum], # JPG
            [pos_info(pos, _block_number) for pos in position if pos.asset.address.checksum == Token('0xaa6e8127831c9de45ae56bb1b0d4d4da6e5665bd').address.checksum], # ETH2x-FLI
            [pos_info(pos, _block_number) for pos in position if pos.asset.address.checksum == Token('0x3Ad707dA309f3845cd602059901E39C4dcd66473').address.checksum], # ETH2x-FLI-P (Polygon)
            # [pos_info(pos, _block_number) for pos in position if pos.asset.address.checksum == Token('0xAbDDAfB225e10B90D798bB8A886238Fb835e2053').address.checksum], # G-UNI-DAI-USDC
            # [pos_info(pos, _block_number) for pos in position if pos.asset.address.checksum == Token('0x0D2A2Df39436b5c5f986552869124bA29b7Df1AC').address.checksum], # G-UNI-icETH-WETH
        ]
    finally:
        context.web3.eth.default_block = default_block

    return poses


In [22]:
poses_20220831 = index_coop_assets(block_20220831)
df_pos = pd.DataFrame([p[0] for p in poses_20220831 if len(p) > 0])
df_pos

USDC
DAI
WETH
cDAI
cUSDC
DATA
GMI
INDEX
icETH
JPG


,0,1,2
0,3007754.3,USDC,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48
1,1463437.5,DAI,0x6b175474e89094c44da98b954eedeac495271d0f
2,16261.0,WETH,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
3,380208.9,cDAI,0x5d3a536e4d6dbd6114cc1ead35777bab948e3643
4,737808.5,cUSDC,0x39aa39c021dfbae8fac545936693ac917d5e7563
5,196.3,DATA,0x33d63ba1e57e54779f7ddaeaa7109349344cf5f1
6,1080.9,GMI,0x47110d43175f7f2c2425e7d15792acc5817eb44f
7,786927.4,INDEX,0x0954906da0bf32d5479e25f46056d22f08464cab
8,0.0,icETH,0x7c07f7abe10ce8e33dc6c5ad68fe033085256a84
9,1063.6,JPG,0x02e7ac540409d32c90bfb51114003a9e1ff0249c


### Obtain LP positions

In [10]:
g_uni_dai_usdc = Token('0xAbDDAfB225e10B90D798bB8A886238Fb835e2053')
default_block = context.web3.eth.default_block
context.web3.eth.default_block = block_20221031

t0 = Token(g_uni_dai_usdc.functions.token0().call())
t1 = Token(g_uni_dai_usdc.functions.token1().call())

output = (g_uni_dai_usdc.functions.gelatoBalance0().call() //
10**t0.decimals *
t0.models.price.dex_db_prefer(return_type=PriceWithQuote,block_number=block_20221031).price, t0.symbol,

g_uni_dai_usdc.functions.gelatoBalance1().call() //
10**t1.decimals *
t1.models.price.dex_db_prefer(return_type=PriceWithQuote,block_number=block_20221031).price, t1.symbol,
)
context.web3.eth.default_block = default_block

display(output)

(3357.1159869267917, 'DAI', 3098.892929698762, 'USDC')

## 2. Streaming

In [4]:
# https://docs.tokensets.com/developers/contracts/deployed/protocol

setv2_BasicIssuanceModule = Contract('0xd8EF3cACe8b4907117a45B0b125c68560532F94D')
setv2_StreamingFeeModule = Contract('0x08f866c74205617B6F3903EF481798EcED10cDEC')
setv2_DebtIssuanceModule = Contract('0x39F024d621367C044BacE2bf0Fb15Fb3612eCB92')

In [24]:
def setv2_fee(_setv_module, _start_block, _end_block, _set_token_addr):
    set_token_addr = Address(_set_token_addr).checksum
    df_mint = pd.DataFrame(
        _setv_module.fetch_events(
            _setv_module.events.SetTokenIssued,
            from_block = _start_block,
            to_block = _end_block,
            argument_filters={'_setToken': set_token_addr},
            contract_address=_setv_module.address,
        )
    )

    df_burn = pd.DataFrame(
        _setv_module.fetch_events(
            _setv_module.events.SetTokenRedeemed,
            from_block = _start_block,
            to_block = _end_block,
            argument_filters={'_setToken': set_token_addr},
            contract_address=_setv_module.address,
        )
    )
    
    if not df_burn.empty:
        df_burn = df_burn.assign(_quantity = lambda r: -1 * r._quantity)
    else:
        df_burn = pd.DataFrame(
            columns=['blockNumber', 'logIndex', 'transactionIndex', '_quantity', '_redeemer'],
            data=[(0, 0, 0, 0, '')]).query('_redeemer != ""')

    df_mint_burn = (pd
        .concat([df_mint, df_burn])
        .reset_index(drop=True)
        .loc[:, ['blockNumber', 'logIndex', 'transactionIndex', '_quantity', '_redeemer']]
        .sort_values(['blockNumber', 'logIndex', 'transactionIndex'])
    )

    return df_mint_burn

In [25]:
dpi_token_addr = Token('0x1494CA1F11D487c2bBe4543E90080AeBa4BA3C2b').address.checksum
setv2_fee(setv2_BasicIssuanceModule, 0, 16_000_000, dpi_token_addr)

,blockNumber,logIndex,transactionIndex,_quantity,_redeemer
0,10836209,132,69,1000000000000000000,NaN
1,10836292,304,123,1041114171118800626,NaN
2,10837157,188,50,1108593800732888550,NaN
3,10842731,108,141,50000000000000000000,NaN
4,10843668,166,72,2803889684007558244,NaN
...,...,...,...,...,...
3368,15992962,64,0,237617730000000000000,NaN
3369,15992996,80,2,75932230000000000000,NaN
3370,15998844,254,104,79634392964281603840,NaN
3371,15999006,64,0,87909210000000000000,NaN


In [26]:
block_20200910 = BlockNumber.from_timestamp(datetime(2020, 9, 10, 23, 59, 59, tzinfo=timezone.utc))
block_20200911 = BlockNumber.from_timestamp(datetime(2020, 9, 12, 23, 59, 59, tzinfo=timezone.utc))-1
block_20200912 = BlockNumber.from_timestamp(datetime(2020, 9, 13, 23, 59, 59, tzinfo=timezone.utc))-1
block_20200919 = BlockNumber.from_timestamp(datetime(2020, 9, 19, 23, 59, 59, tzinfo=timezone.utc))-1
block_20200922 = BlockNumber.from_timestamp(datetime(2020, 9, 22, 23, 59, 59, tzinfo=timezone.utc))-1
block_20200923 = BlockNumber.from_timestamp(datetime(2020, 9, 23, 23, 59, 59, tzinfo=timezone.utc))-1
block_20230116 = BlockNumber.from_timestamp(datetime(2023, 1, 16, 23, 59, 59, tzinfo=timezone.utc))-1

In [27]:
df = setv2_fee(setv2_DebtIssuanceModule, 0, context.block_number, '0x0b498ff89709d3838a063f1dfa463091f9801c2b')
df

,blockNumber,logIndex,transactionIndex,_quantity,_redeemer
0,12377073,83,55,10000000000000000000,NaN
1,12383329,244,126,1000000000000000000,NaN
2,12383787,291,103,1747243767523597251116,NaN
3,12388983,192,83,1000000000000000000,NaN
4,12410924,282,172,143273720303189842535,NaN
...,...,...,...,...,...
1371,16413769,47,1,2005457390000000000000,NaN
1372,16413798,24,0,2092313900000000000000,NaN
2271,16414606,81,59,-5725000000000000000000,0xeA716Ed94964Ed0126Fb2fA3b546eD7F209cC2b8
1373,16428032,23,0,830379400000000000000,NaN


In [51]:
def index_coop_revenue_issue(_token_addr, _setv2_mod, _year, _month, _streaming_rate, _coop_rate, _mint_redeem_rate = 0.0, _coop_mr_rate = 0.0):
    prod_token = Token(_token_addr)
    prod_token_decimals = prod_token.decimals

    days_in_month = pd.Timestamp(year=_year, month=_month, day=1).days_in_month
    prev_eom = datetime(_year, _month, 1, 23, 59, 59, tzinfo=timezone.utc) - timedelta(days=1)
    this_eom = datetime(_year, _month, days_in_month, 23, 59, 59, tzinfo=timezone.utc)
    start_block = BlockNumber.from_timestamp(prev_eom)+1
    end_block = BlockNumber.from_timestamp(this_eom)-1

    df = setv2_fee(_setv2_mod, 0, end_block, _token_addr)

    prev_aum = df.query('blockNumber <= @start_block')['_quantity'].sum()
    prev_end_block = start_block

    row_data = []

    for day in range(1, days_in_month+1):
        blk_eod = BlockNumber.from_timestamp(datetime(_year, _month, day, 23, 59, 59, tzinfo=timezone.utc))-1
        unit = df.query('blockNumber <= @blk_eod')['_quantity'].sum() / 10 ** prod_token_decimals
        mint_redeem_events = df.query('(blockNumber > @prev_end_block) & (blockNumber <= @blk_eod)')['_quantity'] / 10 ** prod_token_decimals
        mint_unit = mint_redeem_events.loc[mint_redeem_events > 0].sum()
        redeem_unit = mint_redeem_events.loc[mint_redeem_events < 0].sum()
        mint_redeem_unit = mint_unit - redeem_unit
        prev_end_block = blk_eod
        try:
            price = prod_token.models(block_number = blk_eod).price.dex_db_prefer()['price']
        except (ModelDataError, ModelRunError):
            try:
                price = prod_token.models(block_number = blk_eod).price.dex_db_ring3()['price'] 
            except (ModelDataError, ModelRunError):
                price = 1

        aum = unit * price
        
        fee = aum * _streaming_rate / 365
        fee_coop = fee * _coop_rate
        fee_method = fee * (1 - _coop_rate)
        
        fee_mint = mint_redeem_unit * price * _mint_redeem_rate
        fee_mint_coop = fee_mint * _coop_mr_rate
        fee_mint_method = fee_mint * (1 - _coop_mr_rate)
        
        row_data.append((blk_eod, day, mint_unit, redeem_unit, mint_redeem_unit, unit, price, aum,
                         fee, fee_coop, fee_method, fee_mint, fee_mint_coop, fee_mint_method))

    df_row = pd.DataFrame(row_data,
        columns = ['block_number', 'day', 'mint_unit', 'redeem_unit', 'mint_redeem_unit', 'unit',
                   'price', 'aum',
                   'fee', 'fee_coop', 'fee_method', 'fee_mint', 'fee_mint_coop', 'fee_mint_method'])
    return df_row

In [ ]:
# total supply of the token != issued + redeemed

try:
    default_block = context.web3.eth.default_block
    context.web3.eth.default_block = 12480784

    display(Token('0x0b498ff89709d3838a063f1dfa463091f9801c2b').total_supply_scaled)
finally:
    context.web3.eth.default_block = default_block

In [87]:
year = 2022
month = 7

In [88]:
df_dpi = index_coop_revenue_issue('0x1494ca1f11d487c2bbe4543e90080aeba4ba3c2b', setv2_BasicIssuanceModule, year, month, 0.0095, 0.7)
df_mvi = index_coop_revenue_issue('0x72e364f2abdc788b7e918bc238b21f109cd634d7', setv2_BasicIssuanceModule, year, month, 0.0095, 1.0)
df_data = index_coop_revenue_issue('0x33d63ba1e57e54779f7ddaeaa7109349344cf5f1', setv2_BasicIssuanceModule, year, month, 0.0095, 0.7)
df_bed = index_coop_revenue_issue('0x2af1df3ab0ab157e1e2ad8f88a7d04fbea0c7dc6', setv2_BasicIssuanceModule, year, month, 0.0025, 0.5)
df_gmi = index_coop_revenue_issue('0x47110d43175f7f2c2425e7d15792acc5817eb44f', setv2_BasicIssuanceModule, year, month, 0.0195, 0.6)

df_eth2x = index_coop_revenue_issue('0xaa6e8127831c9de45ae56bb1b0d4d4da6e5665bd', setv2_DebtIssuanceModule, year, month, 0.0195, 0.6, 0.001, 0.6)
df_btc2x = index_coop_revenue_issue('0x0b498ff89709d3838a063f1dfa463091f9801c2b', setv2_DebtIssuanceModule, year, month, 0.0195, 0.6, 0.001, 0.6)

In [89]:
df_eth2x

,block_number,day,mint_unit,redeem_unit,mint_redeem_unit,unit,price,aum,fee,fee_coop,fee_method,fee_mint,fee_mint_coop,fee_mint_method
0,15059679,1,81472.760397,-85404.785146,166877.545543,1.416710e+06,6.283762,8.902269e+06,475.600668,285.360401,190.240267,1048.618768,629.171261,419.447507
1,15066167,2,45388.364004,-43109.596980,88497.960984,1.418989e+06,6.395818,9.075594e+06,484.860500,290.916300,193.944200,566.016822,339.610093,226.406729
2,15072707,3,26192.913706,-26202.483326,52395.397032,1.418979e+06,6.454609,9.158957e+06,489.314146,293.588488,195.725658,338.191824,202.915095,135.276730
3,15079142,4,34398.260724,-57673.996013,92072.256736,1.395704e+06,7.423895,1.036156e+07,553.562630,332.137578,221.425052,683.534788,410.120873,273.413915
4,15085619,5,51898.363149,-40957.486676,92855.849825,1.406644e+06,7.167163,1.008165e+07,538.608697,323.165218,215.443479,665.513017,399.307810,266.205207
5,15092099,6,41348.600206,-57141.475550,98490.075756,1.390852e+06,7.906615,1.099693e+07,587.507108,352.504265,235.002843,778.723123,467.233874,311.489249
6,15098518,7,32032.301906,-23585.558854,55617.860760,1.399298e+06,8.479008,1.186466e+07,633.865508,380.319305,253.546203,471.584312,282.950587,188.633725
7,15105071,8,32429.822107,-19774.376979,52204.199086,1.411954e+06,8.198292,1.157561e+07,618.422937,371.053762,247.369175,427.985259,256.791156,171.194104
8,15111485,9,12471.455504,-9601.371835,22072.827339,1.414824e+06,8.238347,1.165581e+07,622.707648,373.624589,249.083059,181.843615,109.106169,72.737446
9,15117980,10,21594.725929,-8353.769681,29948.495610,1.428065e+06,7.572232,1.081364e+07,577.714900,346.628940,231.085960,226.776958,136.066175,90.710783


In [86]:
fee_total = 0
fee_coop_total = 0
rows = []
for name, df_prod in zip(['dpi', 'mvi', 'data', 'bed', 'gmi', 'eth2x', 'btc2x'], [df_dpi, df_mvi, df_data, df_bed, df_gmi, df_eth2x, df_btc2x]):
    assert np.isclose(0, ((df_prod.fee + df_prod.fee_mint) - (df_prod.fee_coop + df_prod.fee_mint_coop)  - (df_prod.fee_method + df_prod.fee_mint_method)).abs().mean())

    item_fee_total = (df_prod.fee + df_prod.fee_mint).sum()
    item_fee_coop_total = (df_prod.fee_coop + df_prod.fee_mint_coop).sum()

    fee_total += item_fee_total
    fee_coop_total += item_fee_coop_total

    row = (name, round(item_fee_total, 1), round(item_fee_coop_total,1), round(fee_total,1), round(fee_coop_total, 1))
    rows.append(row)
    # print(row)

pd.DataFrame(columns=['product', 'item_fee_total', 'item_fee_coop_total', 'fee_total', 'fee_coop_total',],data=rows).to_excel('202206.xlsx')

In [39]:
df_eth2x = index_coop_revenue_issue('0xaa6e8127831c9de45ae56bb1b0d4d4da6e5665bd', setv2_DebtIssuanceModule, 2022, 9, 0.0195, 0.6, 0.001, 0.6)

In [ ]:
pd.DataFrame(
    setv2_BasicIssuanceModule.fetch_events(
        setv2_BasicIssuanceModule.events.SetTokenIssued,
        from_block = 13_000_000,
        to_block = 13_010_000,
        argument_filters={'_setToken': '0x1494CA1F11D487c2bBe4543E90080AeBa4BA3C2b'}
    )
)

pd.DataFrame(
    setv2_BasicIssuanceModule.fetch_events(
        setv2_BasicIssuanceModule.events.SetTokenRedeemed,
        from_block = 13_000_000,
        to_block = 13_010_000,
        argument_filters={'_setToken': '0x1494CA1F11D487c2bBe4543E90080AeBa4BA3C2b'}
    )
)


In [ ]:
pd.DataFrame(
    setv2_DebtIssuanceModule.fetch_events(
        setv2_DebtIssuanceModule.events.SetTokenIssued,
        from_block = 13_000_000,
        to_block = 13_010_000,
        argument_filters={'_setToken': Address('0xaa6e8127831c9de45ae56bb1b0d4d4da6e5665bd').checksum}
    )
)

pd.DataFrame(
    setv2_DebtIssuanceModule.fetch_events(
        setv2_DebtIssuanceModule.events.SetTokenRedeemed,
        from_block = 13_000_000,
        to_block = 13_010_000,
        argument_filters={'_setToken': Address('0xaa6e8127831c9de45ae56bb1b0d4d4da6e5665bd').checksum}
    )
)

icETH: 

MNYe

DPI
- 0x1494ca1f11d487c2bbe4543e90080aeba4ba3c2b
- 0.95% = 70% Index Coop, 30% DeFi Pulse

MVI:
- setv2_BasicIssuanceModule
- 0x72e364f2abdc788b7e918bc238b21f109cd634d7
- 0.95% = 100% Index Coop

DATA:
- setv2_BasicIssuanceModule
- 0x33d63ba1e57e54779f7ddaeaa7109349344cf5f1
- 0.95% = 70% Index Coop, 30% DeFi Pulse

BED:
- setv2_BasicIssuanceModule
- 0x2af1df3ab0ab157e1e2ad8f88a7d04fbea0c7dc6
- 0.25% = 50% Index Coop, 50% Bankless

GMI:
- setv2_BasicIssuanceModule???
- 0x47110d43175f7f2c2425e7d15792acc5817eb44f
- 1.95% = 60% Index Coop, 40% Bankless

ETH2x-FLI:
- Debt
- 0xaa6e8127831c9de45ae56bb1b0d4d4da6e5665bd
- Streaming Fee: 1.95% (60% Index Coop, 40% DeFi Pulse)
- Mint/Redeem Fee: 0.1% (60% Index Coop, 40% DeFi Pulse)

BTC2x-FLI: Debt
- 0x0b498ff89709d3838a063f1dfa463091f9801c2b'
- Streaming Fee: 1.95% (60% Index Coop, 40% DeFi Pulse)
- Mint/Redeem Fee: 0.1% (60% Index Coop, 40% DeFi Pulse)

Polygon
- ETH2x-FLI
- iETH-FLI
- BTC2x-FLI
- iBTC-FLI
- MATIC2x-FLI
- iMATIC-FLI


In [63]:
# icETH
0x7C07F7aBe10CE8e33DC6C5aD68FE033085256A84

ic_eth_token = Token('0x7C07F7aBe10CE8e33DC6C5aD68FE033085256A84')
ic_eth_token_decimals = ic_eth_token.decimals

df_mint = pd.DataFrame(
    ic_eth_token.fetch_events(
        ic_eth_token.events.Transfer,
        from_block = 0,
        to_block = 16_001_000,
        argument_filters={'from': '0x0000000000000000000000000000000000000000'},
        contract_address=ic_eth_token.address
    ))

df_burn = pd.DataFrame(
    ic_eth_token.fetch_events(
        ic_eth_token.events.Transfer,
        from_block = 0,
        to_block = 16_001_000,
        argument_filters={'to': '0x0000000000000000000000000000000000000000'},
        contract_address=ic_eth_token.address
    )).loc[:, ['blockNumber', 'logIndex', 'from', 'to', 'value']]

if not df_mint.empty:
    df_mint = df_mint.loc[:, ['blockNumber', 'logIndex', 'from', 'to', 'value']]

if not df_burn.empty:
    df_burn = df_burn.loc[:, ['blockNumber', 'logIndex', 'from', 'to', 'value']].assign(value = lambda r: -1 * r.value)

df_mint_burn = (pd.concat([df_mint, df_burn])
    .sort_values(['blockNumber', 'logIndex']).reset_index(drop=True)
    .assign(value = lambda r, decs=ic_eth_token_decimals: r.value / 10 ** decs))
df_mint_burn.loc[:, 'blockTime'] = df_mint_burn.apply(lambda r: BlockNumber(r['blockNumber']).timestamp_datetime, axis = 1)
df_mint_burn.loc[:, 'blockDate'] = df_mint_burn.blockTime.dt.strftime('%Y-%m-%d')

In [50]:
df_issue = (df_mint_burn.query('`from` == "0x0000000000000000000000000000000000000000"')
    .groupby('blockDate', as_index=False)['value'].sum().rename(columns={'value': 'issue'})
)

df_redeem = (df_mint_burn.query('`to` == "0x0000000000000000000000000000000000000000"')
    .groupby('blockDate', as_index=False)['value'].sum().rename(columns={'value': 'redeem'}).assign(redeem = lambda r: -1 * r.redeem)
)

In [65]:
df_mint_burn

,blockNumber,logIndex,from,to,value,blockTime,blockDate
0,14424402,403,0x0000000000000000000000000000000000000000,0xFafd604d1CC8b6B3B6CC859cF80Fd902972371C1,0.001505,2022-03-20 16:47:02+00:00,2022-03-20
1,14424402,404,0x0000000000000000000000000000000000000000,0xFD75b173c860C1478c3FF39453a0c72feDCE6c81,1.003495,2022-03-20 16:47:02+00:00,2022-03-20
2,14424883,59,0x0000000000000000000000000000000000000000,0xFafd604d1CC8b6B3B6CC859cF80Fd902972371C1,0.000185,2022-03-20 18:30:09+00:00,2022-03-20
3,14424883,60,0x0000000000000000000000000000000000000000,0x37e6365d4f6aE378467b0e24c9065Ce5f06D70bF,0.123346,2022-03-20 18:30:09+00:00,2022-03-20
4,14430129,222,0x0000000000000000000000000000000000000000,0xFafd604d1CC8b6B3B6CC859cF80Fd902972371C1,0.000024,2022-03-21 14:05:17+00:00,2022-03-21
...,...,...,...,...,...,...,...
657,15989604,129,0x0000000000000000000000000000000000000000,0xb39235EBFE6BCcB274cF4d05b80Fd08Ca98BfEE4,19.7,2022-11-17 12:00:23+00:00,2022-11-17
658,15991180,430,0x0000000000000000000000000000000000000000,0xe5d028350093a743A9769e6FD7F5546eEdDAA320,20.0,2022-11-17 17:17:11+00:00,2022-11-17
659,15991257,29,0x0000000000000000000000000000000000000000,0xe5d028350093a743A9769e6FD7F5546eEdDAA320,26.0,2022-11-17 17:32:47+00:00,2022-11-17
660,15996161,52,0x0000000000000000000000000000000000000000,0xe5d028350093a743A9769e6FD7F5546eEdDAA320,14.0,2022-11-18 10:00:23+00:00,2022-11-18


In [73]:
df_aum = (df_mint_burn
    .groupby('blockDate', as_index=False)['value'].sum().rename(columns={'value': 'unit_flow'})
    .assign(unit = lambda x: x['unit_flow'].cumsum()))

df_unit_fee = (df_aum
    .merge(df_issue, how = 'left', on = 'blockDate')
    .merge(df_redeem, how = 'left', on = 'blockDate').fillna(0))

print(df_unit_fee['unit'].to_list()[-1] - df_mint_burn.value.sum())

display(df_unit_fee)

0.0


In [114]:
# icETH
0x7C07F7aBe10CE8e33DC6C5aD68FE033085256A84

df_1 = pd.DataFrame(
    setv2_BasicIssuanceModule.fetch_events(
        setv2_DebtIssuanceModule.events.SetTokenIssued,
        from_block = 0,
        to_block = 13_010_000,
        argument_filters={'_setToken': '0x0b498ff89709d3838a063f1dfa463091f9801c2b'},
        contract_address=setv2_DebtIssuanceModule.address
    )
)

df_2 = pd.DataFrame(
    setv2_BasicIssuanceModule.fetch_events(
        setv2_DebtIssuanceModule.events.SetTokenRedeemed,
        from_block = 0,
        to_block = 13_010_000,
        argument_filters={'_setToken': '0x0b498ff89709d3838a063f1dfa463091f9801c2b'},
        contract_address=setv2_DebtIssuanceModule.address
    )
).assign(_quantity = lambda r: -1 * r._quantity)

pd.concat([df_1, df_2]).loc[:, ['blockNumber', 'logIndex', 'transactionIndex', '_quantity', '_managerFee', '_redeemer']]

,blockNumber,logIndex,transactionIndex,_quantity,_managerFee,_redeemer
0,12377073,83,55,10000000000000000000,10000000000000000,NaN
1,12383329,244,126,1000000000000000000,1000000000000000,NaN
2,12383787,291,103,1747243767523597251116,1747243767523597251,NaN
3,12388983,192,83,1000000000000000000,1000000000000000,NaN
4,12410924,282,172,143273720303189842535,143273720303189842,NaN
...,...,...,...,...,...,...
347,13007355,20,0,-679733364526831180844,679733364526831180,0x00000000000123685885532dcB685c442Dc83126
348,13009406,38,3,-1600683719245490700134,1600683719245490700,0x00000000000123685885532dcB685c442Dc83126
349,13009613,25,1,-1287275985458876417825,1287275985458876417,0x00000000000123685885532dcB685c442Dc83126
350,13009824,211,116,-2970781326685242215378,2970781326685242215,0x75600ed0819b51e4da873E38bcDC8330C14444Cf
